# Redes bayesianas - Resident Evil

In [10]:
import networkx  # Permite trabajar con grafos
import pgmpy.models as pgmm  # Modelos gráficos de probabilidad
import pgmpy.factors as pgmf  # Tablas de probabilidades condicionales y
                              # potenciales de probabilidad
import pgmpy.inference as pgmi  # Inferencia probabilística exacta

In [11]:
import pgmpy.readwrite.XMLBIF as rwBIF

reader = rwBIF.XMLBIFReader('residentevil.xml')
Modelo_residentevil = reader.get_model()

In [68]:
Modelo_residentevil.cpds[11]

<TabularCPD representing P(Cooperativo:2 | Refugio:2, Grupo:2) at 0x10dd96ba8>

In [6]:
Modelo_residentevil.check_model()



/Users/anapareciendo/anaconda/lib/python3.6/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


True

In [80]:
#Función que descarta las variables irrelevantes

def descarta(m, c, e):
    lista = c+e
    descartados = []
    padres = []
    for l in lista:
        subgrafo = Modelo_residentevil.subgraph(networkx.ancestors(Modelo_residentevil, l))
        for s in subgrafo:
            padres.append(s)
                          
    for n in m.nodes():
        if n not in lista and n not in padres:
            descartados.append(n)
                  
    return descartados  

consulta = []
consulta.append("Refugio")
evidencias = []
evidencias.append("Ciudad")
evidencias.append("Cooperativo")

descarta(Modelo_residentevil,consulta,evidencias)

['Horda',
 'Medicina',
 'Agua',
 'Comida',
 'Vehiculo',
 'Armas',
 'Soldados umbrella']

In [95]:
#Función que devuelve los factores iniciales reducidos según las evidencias

def factores_iniciales(m,d,e):
    c = 0
    factores = {}
    for n in m.nodes():
        if n not in d:
            factores[n]= m.cpds[c].to_factor()     
        c+=1
    for k,v in factores.items():
        for i in v.scope():
            if i in e:
                v = v.reduce([(i, e[i])])
        
    return factores

d = descarta(Modelo_residentevil, consulta, evidencias)
e = {'Ciudad':1, 'Cooperativo':1}
factores = factores_iniciales(Modelo_residentevil,d,e)

for clave, valor in factores.items():
    print(clave)
    print(valor.scope())
    print(valor)

Lugar
['Lugar']
╒═════════╤══════════════╕
│ Lugar   │   phi(Lugar) │
╞═════════╪══════════════╡
│ Lugar_0 │       0.3000 │
├─────────┼──────────────┤
│ Lugar_1 │       0.2000 │
├─────────┼──────────────┤
│ Lugar_2 │       0.5000 │
╘═════════╧══════════════╛
Ciudad
['Lugar']
╒═════════╤══════════════╕
│ Lugar   │   phi(Lugar) │
╞═════════╪══════════════╡
│ Lugar_0 │       0.4000 │
├─────────┼──────────────┤
│ Lugar_1 │       0.8000 │
├─────────┼──────────────┤
│ Lugar_2 │       0.2000 │
╘═════════╧══════════════╛
Refugio
['Refugio']
╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.4000 │
├───────────┼────────────────┤
│ Refugio_1 │         0.6000 │
╘═══════════╧════════════════╛
Grupo
['Grupo']
╒═════════╤══════════════╕
│ Grupo   │   phi(Grupo) │
╞═════════╪══════════════╡
│ Grupo_0 │       0.2000 │
├─────────┼──────────────┤
│ Grupo_1 │       0.8000 │
╘═════════╧══════════════╛
Cooperativo
['Grupo', 'Refugio']
╒═════

In [149]:
#Función que elimina las variables ocultas

def eliminar_variables(factores, consultas, evidencias):
    eliminar = []
    res = {}
    for k,v in factores.items():
        if k not in consultas and k not in evidencias:
            eliminar.append(k)
    
    for e in eliminar:
        for k,v in factores.items():
            if e in v.scope() and len(v.scope()) > 1:
                phi_not = pgmf.factor_product(factores[e],v)
                phi_not.marginalize([e])
                factores[k] = phi_not
    for k,v in factores.items():
        for i in v.scope():
            if i in consultas:
                res[k] = v
        
    return res
        

consultas = []
consultas.append("Refugio")
evidencias = []
evidencias.append("Ciudad")
evidencias.append("Cooperativo")

d = descarta(Modelo_residentevil, consultas, evidencias)
e = {'Ciudad':1, 'Cooperativo':1}

factores = factores_iniciales(Modelo_residentevil,d,e)

prueba = eliminar_variables(factores,consultas,evidencias)
          
for k,v in prueba.items():
    print(k)
    print(v.scope())
    print(v)

Refugio
['Refugio']
╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.4000 │
├───────────┼────────────────┤
│ Refugio_1 │         0.6000 │
╘═══════════╧════════════════╛
Cooperativo
['Refugio']
╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.8400 │
├───────────┼────────────────┤
│ Refugio_1 │         0.9200 │
╘═══════════╧════════════════╛


In [150]:
#Función que multiplica y normaliza los factores restantes
def mn_factores(factores):
    m = []
    for k,v in factores.items():
        if not not v.scope():
            m.append(v)
        
    phi = pgmf.factor_product(*m)
    phi.normalize()
    
    return phi

end = mn_factores(eliminar_variables(factores,consultas,evidencias))
print(end)

╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.3784 │
├───────────┼────────────────┤
│ Refugio_1 │         0.6216 │
╘═══════════╧════════════════╛


In [135]:
#Eliminación de variables: Uso de la clase VariableElimination
#Para comprobar que la implementación del algoritmo es correcta
Modelo_re_ev = pgmi.VariableElimination(Modelo_residentevil)
consulta = Modelo_re_ev.query(['Refugio'], {'Ciudad': 1, 'Cooperativo': 1})
print(consulta['Refugio'])

╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.3784 │
├───────────┼────────────────┤
│ Refugio_1 │         0.6216 │
╘═══════════╧════════════════╛


/Users/anapareciendo/anaconda/lib/python3.6/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)
